In [1]:
import os

SEASONS = list(range(2023,2025))

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [2]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
# Make sure to install playwright browsers by running playwright install on the command line or !playwright install from Jupyter

In [3]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [4]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [5]:
#for season in SEASONS:
#    await scrape_season(season)

In [6]:
standings_files = os.listdir(STANDINGS_DIR)

In [12]:
async def scrape_game(standings_file):
    # Lê o arquivo HTML com a codificação padrão
    with open(standings_file, 'r', encoding='utf-8') as f:
        html = f.read()

    # Analisa o conteúdo HTML
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    # Processa cada box score
    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        
        # Ignora se o arquivo já existe
        if os.path.exists(save_path):
            continue

        # Obtém o conteúdo HTML da página
        html = await get_html(url, "#content")
        if not html:
            continue

        # Salva o conteúdo com codificação UTF-8
        with open(save_path, "w+", encoding="utf-8") as f:
            f.write(html)

NameError: name 'box_scores' is not defined

In [11]:
import pandas as pd

for season in SEASONS:
    files = [s for s in standings_files if str(season) in s]
    
    for f in files:
        filepath = os.path.join(STANDINGS_DIR, f)
        
        await scrape_game(filepath)